# Building a multi-tool agent with a UI

## Imports

In [4]:
from smolagents import LiteLLMModel, Tool, CodeAgent
from ddgs import DDGS
from dotenv import load_dotenv
import huggingface_hub
import requests
import gradio as gr

## Backend

In [ ]:
from smolagents import LiteLLMModel

if os.getenv('AGENT_BACKEND')=='ollama':
    model = LiteLLMModel(
        model_id="ollama_chat/qwen2:7b",     # Any Ollama-supported chat model
        api_base="http://127.0.0.1:11434",   # Ollama’s OpenAI-compatible endpoint
        num_ctx=4096,                        # Adjust per model capability
    )
elif os.getenv('AGENT_BACKEND')=='hf':
    # TODO : Implement HF Inference
    pass

messages = [
  {"role": "user", "content": [{"type": "text", "text": "Hello, how are you?"}]}
]

print(model(messages))

ChatMessage(role='assistant', content="Hello! I'm just a computer program, so I don't have feelings or emotions like humans do. I exist to provide information and assist with tasks to the best of my abilities. How can I help you today?", tool_calls=None, raw=ModelResponse(id='chatcmpl-4d8ed0c7-d8c1-40a0-b58c-77ccc738d365', created=1768211261, model='ollama_chat/qwen2:7b', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! I'm just a computer program, so I don't have feelings or emotions like humans do. I exist to provide information and assist with tasks to the best of my abilities. How can I help you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=45, prompt_tokens=25, total_tokens=70, completion_tokens_details=None, prompt_tokens_details=None)), token_usage=TokenUsage(input_tokens=25, output_tokens=45, total_tokens=70))


d:\Coding\DI\190_W10_D2\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="Hello! I...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


## 3. Tools 

In [16]:
from smolagents import LiteLLMModel, Tool, CodeAgent
from ddgs import DDGS

class CurrentTimeTool(Tool):
    name = "current_time"
    description = "Returns the current time in Tel-Aviv."
    inputs = {}
    output_type = "string"

    def forward(self) -> str:
        from datetime import datetime
        import pytz

        tz = pytz.timezone('Israel')
        ct = datetime.now(tz)
        return ct

class HttpGetTool(Tool):
    name = "http_get"
    description = "Sends an HTTP GET requests and returns the response."
    inputs = {"url": {"type": "string", "description": "URL to query"}}
    output_type = "string"

    def forward(self, url: str) -> str:
        import requests
        r = requests.get(url = url)
        return r

# TODO : Add a new custom tool, for example one that could create/write in a file

## 4. Agent

In [19]:
agent = CodeAgent(
    tools=[CurrentTimeTool(), HttpGetTool()],
    model=model,
    max_steps=4,
    name="ResearchAgent",
)

print(agent.run("Make sure you use the provided tools when necessary. Tell me what time it is in Tel Aviv and what is the capital of israel based on this : https://www.iir.cz/en/kontroverzni-otazka-jeruzalema-z-pohledu-mezinarodniho-prava-2"))

╭──────────────────────────────────────────── New run - ResearchAgent ────────────────────────────────────────────╮
│                                                                                                                 │
│ Make sure you use the provided tools when necessary. Tell me what time it is in Tel Aviv and what is the        │
│ capital of israel based on this :                                                                               │
│ https://www.iir.cz/en/kontroverzni-otazka-jeruzalema-z-pohledu-mezinarodniho-prava-2                            │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  current_time_in_tel_aviv = current_time()                                                                        
  print("The current time in Tel Aviv is:", current_time_in_tel_aviv)                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

d:\Coding\DI\190_W10_D2\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='Thought:...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Execution logs:
The current time in Tel Aviv is: 2026-01-12 11:59:04.785855+02:00

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The current time in Tel Aviv is 2026-01-12 11:59:04.785855+02:00 and the capital of Israel is      
  Jerusalem.")                                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The current time in Tel Aviv is 2026-01-12 11:59:04.785855+02:00 and the capital of Israel is 
Jerusalem.

[Step 4: Duration 2.20 seconds| Input tokens: 9,998 | Output tokens: 313]

The current time in Tel Aviv is 2026-01-12 11:59:04.785855+02:00 and the capital of Israel is Jerusalem.


## 6. Gradio UI

In [ ]:
# TODO Create the Gradio UI